# Time Series Forecasting – Energy Load with External Regressors

This notebook is a complete mini-project for **time series forecasting** in an
energy / traffic setting. It is structured in three tiers:

1. **Tier 1 – Analysis**: EDA, decomposition, anomaly detection, cross-correlation.
2. **Tier 2 – Modelling**: ARIMA/SARIMA-type models + ML regressors with exogenous variables and proper evaluation.
3. **Tier 3 – Advanced / Deployment**: Probabilistic forecasts and a simple champion–challenger setup.

The notebook assumes you have an **hourly (or daily) energy load dataset** with an
external regressor such as temperature. You can adapt the loading cell to your
own data.


## 0. Setup and Data Loading

Expected data format (you can adjust this to your dataset):

- CSV file under `data/energy.csv`.
- Columns:
  - `timestamp` – Datetime string.
  - `load` – Energy load (target).
  - `temperature` – External regressor (optional but recommended).

If your dataset has different names, you can change the column references in
this section.


In [ ]:
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from sklearn.ensemble import GradientBoostingRegressor

import statsmodels.api as sm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.statespace.sarimax import SARIMAX

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 5)

DATA_PATH: Path = Path("data") / "energy.csv"
RANDOM_STATE: int = 42
np.random.seed(RANDOM_STATE)

if not DATA_PATH.exists():
    raise FileNotFoundError(
        f"Expected data at {DATA_PATH.resolve()} – please place your CSV there "
        "or edit DATA_PATH."
    )

df = pd.read_csv(DATA_PATH)

# Basic sanity checks and parsing
expected_cols = {"timestamp", "load"}
if not expected_cols.issubset(df.columns):
    raise ValueError(
        f"Dataframe must contain at least columns {expected_cols}, got {set(df.columns)}"
    )

df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")
df = df.sort_values("timestamp").dropna(subset=["timestamp", "load"])
df = df.set_index("timestamp").asfreq("H")  # adjust to 'D' for daily data

df[["load"]].head()

### Helper metrics

We will use RMSE, MAE and MAPE throughout the notebook.


In [ ]:
def rmse(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Compute root mean squared error.

    Parameters
    ----------
    y_true : np.ndarray
        True target values.
    y_pred : np.ndarray
        Predicted target values.

    Returns
    -------
    float
        RMSE value.
    """
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))


def mape(y_true: np.ndarray, y_pred: np.ndarray) -> float:
    """Compute mean absolute percentage error (MAPE).

    Notes
    -----
    Very small true values are clipped to avoid division by zero.
    """
    y_true_safe = np.clip(y_true, 1e-6, None)
    return float(np.mean(np.abs((y_true - y_pred) / y_true_safe)) * 100.0)


y_example = df["load"].to_numpy()
print("Example RMSE self-check:", rmse(y_example, y_example))

## Tier 1 – Time Series Analysis

Goals:

- Visualise the series.
- Decompose into trend, seasonality and residual.
- Detect anomalies with simple rules and statistical thresholds.
- Explore cross-correlation with an external regressor (e.g. temperature).


### 1.1 Basic EDA and decomposition


In [ ]:
# Plot the full time series
df["load"].plot()
plt.title("Energy load over time")
plt.ylabel("Load")
plt.show()

# Seasonal decomposition (adjust period to your data)
period = 24  # e.g. daily seasonality for hourly data
decomp = seasonal_decompose(df["load"].dropna(), model="additive", period=period)
fig = decomp.plot()
fig.set_size_inches(10, 8)
plt.tight_layout()
plt.show()

df["trend"] = decomp.trend
df["seasonal"] = decomp.seasonal
df["resid"] = decomp.resid
df[["load", "trend", "seasonal", "resid"]].head()

### 1.2 Simple anomaly detection

We apply two simple approaches:

- **Statistical residual-based**: flag points where residuals exceed a
  multiple of the residual standard deviation.
- **Rule-based**: flag values below zero or above a high quantile.


In [ ]:
# Statistical anomaly detection based on residuals
resid = df["resid"].dropna()
resid_std = resid.std()
z_thresh = 3.0

stat_anomaly_mask = np.abs(resid) > z_thresh * resid_std
stat_anomalies = resid[stat_anomaly_mask]

# Rule-based anomalies (negative load or extremely high)
upper_threshold = df["load"].quantile(0.99)
rule_anomaly_mask = (df["load"] < 0) | (df["load"] > upper_threshold)
rule_anomalies = df["load"][rule_anomaly_mask]

print("Statistical anomalies:", stat_anomalies.shape[0])
print("Rule-based anomalies: ", rule_anomalies.shape[0])

# Visualise anomalies
plt.figure(figsize=(12, 5))
plt.plot(df.index, df["load"], label="load", alpha=0.7)
plt.scatter(stat_anomalies.index, stat_anomalies.index.map(df["load"]),
            color="red", label="statistical", s=15)
plt.scatter(rule_anomalies.index, rule_anomalies.values,
            color="orange", label="rule-based", s=15)
plt.legend()
plt.title("Detected anomalies")
plt.ylabel("Load")
plt.show()

### 1.3 Cross-correlation with external regressors

If the dataset includes a `temperature` column, we can inspect the
relationship between temperature and load, including lagged effects.


In [ ]:
if "temperature" in df.columns:
    # Basic scatter and joint plot
    sns.scatterplot(x="temperature", y="load", data=df.sample(min(2000, len(df))))
    plt.title("Load vs Temperature (sample)")
    plt.show()

    # Cross-correlation function (simplified)
    # We consider a range of lags and compute correlation coefficients.
    max_lag = 48  # two days for hourly data
    load_vals = df["load"].fillna(method="ffill").to_numpy()
    temp_vals = df["temperature"].fillna(method="ffill").to_numpy()

    lags = np.arange(-max_lag, max_lag + 1)
    ccs: List[float] = []
    for lag in lags:
        if lag < 0:
            x = temp_vals[:lag]
            y = load_vals[-lag:]
        elif lag > 0:
            x = temp_vals[lag:]
            y = load_vals[:-lag]
        else:
            x = temp_vals
            y = load_vals
        ccs.append(np.corrcoef(x, y)[0, 1])

    plt.plot(lags, ccs)
    plt.axhline(0, color="black", linewidth=0.8)
    plt.xlabel("Lag (hours, temp leads if negative)")
    plt.ylabel("Cross-correlation")
    plt.title("Cross-correlation between temperature and load")
    plt.show()
else:
    print("No 'temperature' column found – skipping cross-correlation analysis.")

## Tier 2 – Modelling

Goals:

- Train standard time series models:
  - SARIMAX (ARIMA with seasonality and exogenous variables).
  - Machine-learning regressor with lagged features and exogenous vars.
- Use robust evaluation:
  - Rolling-origin cross-validation.
  - Error distribution analysis.


### 2.1 Train / test split

We keep the last chunk of the series as a hold-out test set. The earlier
part is used for model fitting and cross-validation.


In [ ]:
horizon = 24 * 7  # one week of hourly data for test set; adjust as needed
if len(df) <= horizon * 2:
    raise ValueError("Not enough data relative to chosen horizon.")

train_df = df.iloc[:-horizon].copy()
test_df = df.iloc[-horizon:].copy()

print("Train size:", train_df.shape[0])
print("Test size: ", test_df.shape[0])

train_df[["load"]].tail(), test_df[["load"]].head()

### 2.2 SARIMAX model (with optional exogenous regressor)


In [ ]:
def fit_sarimax(
    train: pd.DataFrame,
    test: pd.DataFrame,
    order: Tuple[int, int, int] = (1, 0, 1),
    seasonal_order: Tuple[int, int, int, int] = (1, 1, 1, 24),
) -> Tuple[np.ndarray, np.ndarray]:
    """Fit a SARIMAX model and forecast on the test horizon.

    Parameters
    ----------
    train : pd.DataFrame
        Training set with column `load` and optionally `temperature`.
    test : pd.DataFrame
        Test set with the same columns. Only index length is used.
    order : tuple
        Non-seasonal ARIMA order (p, d, q).
    seasonal_order : tuple
        Seasonal ARIMA order (P, D, Q, s).

    Returns
    -------
    Tuple[np.ndarray, np.ndarray]
        Tuple of (fitted_values on train, forecast_values on test).
    """
    endog_train = train["load"].astype(float)
    exog_cols: List[str] = []
    if "temperature" in train.columns:
        exog_cols = ["temperature"]

    exog_train = train[exog_cols] if exog_cols else None
    exog_test = test[exog_cols] if exog_cols else None

    model = SARIMAX(
        endog_train,
        exog=exog_train,
        order=order,
        seasonal_order=seasonal_order,
        enforce_stationarity=False,
        enforce_invertibility=False,
    )
    res = model.fit(disp=False)

    fitted = res.fittedvalues.to_numpy()
    forecast = res.forecast(steps=len(test), exog=exog_test).to_numpy()
    return fitted, forecast


sarimax_fitted, sarimax_forecast = fit_sarimax(train_df, test_df)

print("SARIMAX train RMSE:", rmse(train_df["load"].to_numpy(), sarimax_fitted))
print("SARIMAX test RMSE: ", rmse(test_df["load"].to_numpy(), sarimax_forecast))

plt.plot(train_df.index, train_df["load"], label="train")
plt.plot(test_df.index, test_df["load"], label="test")
plt.plot(test_df.index, sarimax_forecast, label="sarimax forecast")
plt.legend()
plt.title("SARIMAX forecast vs actual")
plt.show()

### 2.3 Feature engineering for ML regressor

We now build a supervised learning dataset with:

- Lagged load values.
- Calendar features: hour of day, day of week.
- External regressor (temperature).


In [ ]:
def make_lagged_features(
    df_in: pd.DataFrame,
    n_lags: int = 24,
) -> pd.DataFrame:
    """Create a feature matrix with lagged load and calendar features.

    Parameters
    ----------
    df_in : pd.DataFrame
        Input time series with index as timestamp and columns including `load`.
    n_lags : int
        Number of lagged `load` features to include.

    Returns
    -------
    pd.DataFrame
        Feature DataFrame aligned with the original index (rows with
        insufficient history are dropped).
    """
    df_feat = df_in.copy()

    # Lag features
    for lag in range(1, n_lags + 1):
        df_feat[f"lag_{lag}"] = df_feat["load"].shift(lag)

    # Calendar features
    df_feat["hour"] = df_feat.index.hour
    df_feat["dayofweek"] = df_feat.index.dayofweek

    # Keep temperature if present
    if "temperature" in df_feat.columns:
        df_feat["temperature"] = df_feat["temperature"].interpolate(limit_direction="both")

    df_feat = df_feat.dropna()
    return df_feat


lagged_df = make_lagged_features(df, n_lags=24)
lagged_df.head()

In [ ]:
# Align train/test indices with lagged features
lagged_train = lagged_df.loc[train_df.index.intersection(lagged_df.index)]
lagged_test = lagged_df.loc[test_df.index.intersection(lagged_df.index)]

feature_cols = [c for c in lagged_train.columns if c != "load"]
X_train = lagged_train[feature_cols].to_numpy()
y_train = lagged_train["load"].to_numpy()
X_test = lagged_test[feature_cols].to_numpy()
y_test = lagged_test["load"].to_numpy()

X_train.shape, X_test.shape

### 2.4 Rolling-origin cross-validation for the ML model

We use `TimeSeriesSplit` to approximate rolling-origin evaluation.


In [ ]:
def evaluate_ts_model_cv(
    X: np.ndarray,
    y: np.ndarray,
    n_splits: int = 5,
) -> pd.DataFrame:
    """Evaluate a scikit-learn regressor with time series CV.

    A GradientBoostingRegressor is used for illustration.
    """
    tscv = TimeSeriesSplit(n_splits=n_splits)
    metrics: List[Dict[str, float]] = []

    for fold, (train_idx, val_idx) in enumerate(tscv.split(X), start=1):
        X_tr, X_val = X[train_idx], X[val_idx]
        y_tr, y_val = y[train_idx], y[val_idx]

        model = GradientBoostingRegressor(
            random_state=RANDOM_STATE,
            n_estimators=300,
            learning_rate=0.05,
            max_depth=3,
        )
        model.fit(X_tr, y_tr)
        y_hat = model.predict(X_val)

        metrics.append(
            {
                "fold": fold,
                "rmse": rmse(y_val, y_hat),
                "mae": mean_absolute_error(y_val, y_hat),
                "mape": mape(y_val, y_hat),
            }
        )

    return pd.DataFrame(metrics)


cv_metrics = evaluate_ts_model_cv(X_train, y_train, n_splits=5)
cv_metrics

In [ ]:
sns.boxplot(data=cv_metrics.melt(id_vars="fold", value_vars=["rmse", "mape"]),
            x="variable", y="value")
plt.title("Rolling-origin CV error distribution")
plt.ylabel("Error")
plt.show()

### 2.5 Fit final ML model and evaluate on test


In [ ]:
gbr_model = GradientBoostingRegressor(
    random_state=RANDOM_STATE,
    n_estimators=300,
    learning_rate=0.05,
    max_depth=3,
)
gbr_model.fit(X_train, y_train)

y_pred_test = gbr_model.predict(X_test)

print("GBR test RMSE:", rmse(y_test, y_pred_test))
print("GBR test MAE: ", mean_absolute_error(y_test, y_pred_test))
print("GBR test MAPE:", mape(y_test, y_pred_test))

plt.plot(lagged_test.index, y_test, label="actual")
plt.plot(lagged_test.index, y_pred_test, label="gbr forecast")
plt.legend()
plt.title("Gradient boosting forecast vs actual")
plt.show()

sns.histplot(y_test - y_pred_test, bins=30)
plt.title("GBR residual distribution")
plt.xlabel("Error (y_true - y_pred)")
plt.show()

## Tier 3 – Probabilistic Forecasts and Champion–Challenger

Goals:

- Provide **uncertainty estimates** around forecasts.
- Set up a simple **champion–challenger** comparison between models.


### 3.1 Quantile Gradient Boosting (probabilistic forecasts)

We fit separate GradientBoostingRegressor models to estimate quantiles
of the conditional distribution of `load` given features. For simplicity
we use three quantiles: 0.1, 0.5, 0.9.


In [ ]:
quantiles = [0.1, 0.5, 0.9]
quantile_models: Dict[float, GradientBoostingRegressor] = {}

for q in quantiles:
    model_q = GradientBoostingRegressor(
        loss="quantile",
        alpha=q,
        random_state=RANDOM_STATE,
        n_estimators=300,
        learning_rate=0.05,
        max_depth=3,
    )
    model_q.fit(X_train, y_train)
    quantile_models[q] = model_q

quantile_forecasts = {q: m.predict(X_test) for q, m in quantile_models.items()}

plt.figure(figsize=(12, 5))
idx = lagged_test.index
plt.plot(idx, y_test, label="actual", color="black")
plt.plot(idx, quantile_forecasts[0.5], label="q0.5", linestyle="--")
plt.fill_between(idx,
                 quantile_forecasts[0.1],
                 quantile_forecasts[0.9],
                 alpha=0.3,
                 label="q0.1–q0.9 band")
plt.legend()
plt.title("Probabilistic forecast with quantile gradient boosting")
plt.show()

### 3.2 Simple conformal-style intervals from residuals

As a lightweight alternative, we can build intervals by:

- Computing residuals on a validation set.
- Taking empirical quantiles of |residual|.
- Expanding point forecasts by that amount.


In [ ]:
# Here we reuse the CV folds to approximate residual distribution.
tscv = TimeSeriesSplit(n_splits=5)
all_abs_residuals: List[float] = []

for train_idx, val_idx in tscv.split(X_train):
    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]
    m = GradientBoostingRegressor(
        random_state=RANDOM_STATE,
        n_estimators=200,
        learning_rate=0.05,
        max_depth=3,
    )
    m.fit(X_tr, y_tr)
    y_hat_val = m.predict(X_val)
    all_abs_residuals.extend(np.abs(y_val - y_hat_val))

all_abs_residuals_arr = np.array(all_abs_residuals)
alpha = 0.1  # 90% interval
q_conformal = np.quantile(all_abs_residuals_arr, 1 - alpha)
print("Conformal absolute error quantile (90% band):", q_conformal)

point_forecast = y_pred_test
lower_band = point_forecast - q_conformal
upper_band = point_forecast + q_conformal

plt.figure(figsize=(12, 5))
plt.plot(lagged_test.index, y_test, label="actual", color="black")
plt.plot(lagged_test.index, point_forecast, label="point forecast", linestyle="--")
plt.fill_between(lagged_test.index, lower_band, upper_band, alpha=0.3, label="conformal band")
plt.legend()
plt.title("Point forecast with conformal-style prediction band")
plt.show()

### 3.3 Champion–challenger comparison

We compare three candidates on the same test horizon:

- **Naïve**: last observed value carried forward.
- **SARIMAX**: from Tier 2.
- **GBR**: ML regressor with lagged features.

You can extend this table with additional models.


In [ ]:
# Align lengths for fair comparison
y_test_sarimax = test_df["load"].to_numpy()

# Naive forecast: last observed load from train repeated over horizon
naive_forecast = np.repeat(train_df["load"].iloc[-1], len(y_test_sarimax))

champion_rows = []

def model_metrics(name: str, y_true: np.ndarray, y_pred: np.ndarray) -> Dict[str, float]:
    return {
        "model": name,
        "rmse": rmse(y_true, y_pred),
        "mae": mean_absolute_error(y_true, y_pred),
        "mape": mape(y_true, y_pred),
    }

champion_rows.append(model_metrics("NaiveLast", y_test_sarimax, naive_forecast))
champion_rows.append(model_metrics("SARIMAX", y_test_sarimax, sarimax_forecast))

# For GBR, y_test comes from lagged_test; align with test_df tail
aligned_len = min(len(y_test_sarimax), len(y_test))
champion_rows.append(
    model_metrics(
        "GBR",
        y_test[-aligned_len:],
        y_pred_test[-aligned_len:],
    )
)

champion_df = pd.DataFrame(champion_rows).set_index("model")
champion_df

In [ ]:
champion_df.sort_values("rmse").plot(kind="bar")
plt.title("Champion–challenger comparison (lower is better)")
plt.ylabel("Error")
plt.xticks(rotation=0)
plt.show()

In a production setting you would:

- Declare the current best model as **champion**.
- Introduce new models as **challengers**.
- Continuously evaluate them on rolling windows.
- Promote challengers to champion when they show consistent uplift
  under your business metrics (not only RMSE/MAPE).


## 4. Next Steps

- Replace the placeholder dataset with your real energy or traffic data.
- Tune SARIMAX orders and GBR hyperparameters.
- Add more exogenous variables (calendar events, holidays, pricing, etc.).
- Wrap the final model into a small API or batch scoring pipeline for deployment.

This notebook gives you a full path from **EDA** to **probabilistic forecasts**
and a simple **champion–challenger** framework for time series forecasting.
